# Planning-Lab Lesson 3: Markov Decision Process

In the third session we will work on the Markov decision process (MDP)

## Lava environments
The environments used are LavaFloor (visible in the figure) and its variations.

![Lava](images/lava.png)

The agent starts in cell $(0, 0)$ and has to reach the treasure in $(2, 3)$. In addition to the walls of the previous environments, the floor is covered with lava, there is a black pit of death.

Moreover, the agent can't comfortably perform its actions that instead have a stochastic outcome (visible in the figure):

![Dynact](images/dynact.png)

The action dynamics is the following:
- $P(0.8)$ of moving **in the desired direction**
- $P(0.1)$ of moving in a direction 90° with respect to the desired direction

Finally, since the floor is covered in lava, the agent receives a negative reward for each of its steps!

- -0.04 for each lava cell (L)
- -5 for the black pit (P). End of episode
- +1 for the treasure (G). End of episode

In [1]:
import os, sys, random
module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
from timeit import default_timer as timer
from tqdm import tqdm as tqdm

### Environment Properties 

In addition to the varables of the environments you have been using in the previous sessions, there are also a few more:

- $T$: matrix of the transition function $T(s, a, s') \rightarrow [0, 1]$
- $RS$: matrix of the reward function $R(s) \rightarrow \mathbb{R}$

The available actions are still Left, Right, Up, Down.

#### Code Hints:

In [2]:
env = gym.make("LavaFloor-v0")

current_state = env.pos_to_state(0, 0)
next_state = env.pos_to_state(0, 1)
goal_state = env.pos_to_state(2, 3)

print("Number of actions: ", env.action_space.n)
print("Actions: ", env.actions)
print("Reward of starting state:", env.RS[current_state])
print("Reward of goal state:", env.RS[goal_state])
print("Probability from (0, 0) to (0, 1) with action right:", env.T[current_state, 1, next_state])
print("Probability from (0, 0) to (2, 3) with action right:", env.T[current_state, 1, goal_state])

Number of actions:  4
Actions:  {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
Reward of starting state: -0.04
Reward of goal state: 1.0
Probability from (0, 0) to (0, 1) with action right: 0.8
Probability from (0, 0) to (2, 3) with action right: 0.0


In [3]:
env_name = "LavaFloor-v0"
env = gym.make(env_name)

c=0
state_right = env.pos_to_state(0, 1) #state to the tight of start state
for i in range(1,101):
    current_state = env.pos_to_state(0, 0)
    state = env.sample(current_state, 1) #trying to go right
    if (state==state_right): 
        c+=1 #counting how many times the agent reaches the state to the right
        
#computing percentage of time agent reached the state to the right going right, should be around 80%           
print("percentage of time agent reaches the state to the right: ", c/i*100) 

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
        print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])
        print("P for Pozzo, G for Goal")
                    


percentage of time agent reaches the state to the right:  76.0
Transition model for  LavaFloor-v0  : 
probability of reaching  (0, 1) from  (0, 0)  executing action  L  :  0.0
probability of reaching  (0, 1) from  (0, 0)  executing action  R  :  0.8
probability of reaching  (0, 1) from  (0, 0)  executing action  U  :  0.1
probability of reaching  (0, 1) from  (0, 0)  executing action  D  :  0.1
Reward for non terminal states:  -0.04
Reward for state : (1, 3)  (state type:  P ) :  -5.0
P for Pozzo, G for Goal
Reward for state : (2, 3)  (state type:  G ) :  1.0
P for Pozzo, G for Goal


## Assignment 1: Value Iteration Algorithm

Your first assignment is to implement the Value Iteration algorithm on LavaFloor. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state.  You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*, *NiceLavaFloor* and *VeryBadLavaFloor*.

<img src="images/value-iteration.png" width="600">

The *value_iteration* function has to be implemented. Notice that the value iteration approach return a matrix with the value for eacht state, the function *values_to_policy* automatically convert this matrix in the policy.

In [42]:
#Given a state S, returns the max of a list where each element is a sum of the utilities of the states reached with each action from S, 
#multiplied by the probability of the action 
def maxOfActions(currentState, environment, U):

    #print("env.actions:")
    #for action in environment.actions:
    #    print ("action: ", action)

    #print("env.action_space: ", environment.action_space)

    if environment.grid[currentState] == "P" or environment.grid[currentState] == "G":
        return 0
    
    res = list(range(environment.action_space.n)) #list of length of number of actions possible

    #debug:
    #environment.T[environment.pos_to_state(2, 0), ]

    for action in range(environment.action_space.n): #this is the outer loop, correct

        #now I need a inner loop for each other possible state

        val = 0
        
        for nextState in environment.staterange: #This iterate through all states (including the one not reachable because distant more than 1 cell)
      
            # s' = nextState, s = currentState, a = action
            # P(s' | s, a) * U[s']        
            prob = environment.T[currentState, action, nextState] # P(s' | s, a)
            val += prob * U[nextState] # U[s']
        
            #print(f"prob from state {environment.state_to_pos(currentState)} with action {environment.actions[action]} to state {environment.state_to_pos(nextState)} is {prob}")
            
        res[action] = val #result is indexed by action index

    #max(P(s' | s, a) * U[s'])
    return max(res)

def value_iteration(environment, maxiters=300, discount=0.5, max_error=1e-3):
    """
    Performs the value iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: timeout for the iterations
        discount: gamma value, the discount factor for the Bellman equation
        max_error: the maximum error allowd in the utility of any state
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """

    print("using discount", discount)
    
    U_1 = [0 for _ in range(environment.observation_space.n)] # vector of utilities for states S
    U = U_1.copy()
    #
    # Code Here!
    #

    i = 0

    #this is the repeat-until in the pseudo code
    while True:

        U = U_1.copy()
        delta = 0 # maximum change in the utility o any state in an iteration

        #for s in environment

        for s in environment.staterange:
                          
            U_1[s] = environment.RS[s] + discount * maxOfActions(s, environment, U)

            deltaU = abs(U_1[s] - U[s])
            
            if deltaU > delta:
                delta = deltaU

        #print("obs_space", env.observation_space)
        #print("obs_space.n", env.observation_space.n)      
        #print("states range: ", env.staterange)

        #for s in environment.staterange:
        #    print("state: ", environment.state_to_pos(s))
                #break

        if delta < max_error * (1 - discount) / discount:
            print("Exiting with delta: ", delta)
            break
        
        #CODE BELOW IS TO PREVENT INFINITE LOOPS
        i += 1
        
        if i >= maxiters:
            print("BLAM! maxiters REACHED")
            break

    for u in range(0, len(U_1)):
        print(f"Utility at pos {environment.state_to_pos(u)} is {U_1[u]}")
    
    return values_to_policy(np.asarray(U), env) # automatically convert the value matrix U to a policy

**The following code executes Value Iteration and prints the resulting policy**

In [43]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"
#env_name = "NiceLavaFloor-v0"
#env_name = "VeryBadLavaFloor-v0"

env = gym.make(env_name)
print("\nENV RENDER:")
env.render()

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])

t = timer()
policy = value_iteration(env)

print("\nEXECUTION TIME: \n{}".format(round(timer() - t, 4)))
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
results = CheckResult_L3(env_name, policy_render)
results.check_value_iteration()


ENV RENDER:
[['S' 'L' 'L' 'L']
 ['L' 'W' 'L' 'P']
 ['L' 'L' 'L' 'G']]
Transition model for  LavaFloor-v0  : 
probability of reaching  (0, 1) from  (0, 0)  executing action  L  :  0.0
probability of reaching  (0, 1) from  (0, 0)  executing action  R  :  0.8
probability of reaching  (0, 1) from  (0, 0)  executing action  U  :  0.1
probability of reaching  (0, 1) from  (0, 0)  executing action  D  :  0.1
Reward for non terminal states:  -0.04
Reward for state : (1, 3)  (state type:  P ) :  -5.0
Reward for state : (2, 3)  (state type:  G ) :  1.0
using discount 0.5
Exiting with delta:  0.0009554359999999901
Utility at pos (0, 0) is -0.063568364
Utility at pos (0, 1) is -0.072960524
Utility at pos (0, 2) is -0.06394412
Utility at pos (0, 3) is -0.078441776
Utility at pos (1, 0) is -0.041249867999999995
Utility at pos (1, 1) is 0.0
Utility at pos (1, 2) is -0.0407390885
Utility at pos (1, 3) is -5.0
Utility at pos (2, 0) is 0.007469268000000008
Utility at pos (2, 1) is 0.12300467200000001
U

## Assignment 2: Policy Iteration Algorithm (<span style="color:red">*OPTIONAL*</span>)

Your <span style="color:red"> *optional*</span> assignment is to implement the Policy Iteration algorithm on LavaFloor. The solution returned by your algorithm must be a 1-d array of action identifiers where the $i$-th action refers to the $i$-th state. You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*, *NiceLavaFloor* and *VeryBadLavaFloor*.

<img src="images/policy-iteration.png" width="600">

For the *policy evaluation step*, it is necessary to implement this function:

<img src="images/policy-evaluating.png" width="500">

**The following function has to be implemented:**

In [ ]:
def policy_iteration(environment, maxiters=150, discount=0.9, maxviter=10):
    """
    Performs the policy iteration algorithm for a specific environment
    
    Args:
        environment: OpenAI Gym environment
        maxiters: timeout for the iterations
        discount: gamma value, the discount factor for the Bellman equation
        
    Returns:
        policy: 1-d dimensional array of action identifiers where index `i` corresponds to state id `i`
    """
    
    policy = [0 for _ in range(environment.observation_space.n)] #initial policy
    U = [0 for _ in range(environment.observation_space.n)] #utility array

    # Step (1): Policy Evaluation
    #
    # Code Here!
    #
    
    # Step (2) Policy Improvement
    unchanged = True  
    #
    # Code Here!
    #
    
    return np.asarray(policy)

**The following code executes and Value Iteration and prints the resulting policy**

In [ ]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"
#env_name = "NiceLavaFloor-v0"
#env_name = "VeryBadLavaFloor-v0"

env = gym.make(env_name)
print("\nENV RENDER:")
env.render()

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])

t = timer()
policy = policy_iteration(env)

print("\nEXECUTION TIME: \n{}".format(round(timer() - t, 4)))
policy_render = np.vectorize(env.actions.get)(policy.reshape(env.rows, env.cols))
results = CheckResult_L3(env_name, policy_render)
results.check_policy_iteration()

## Comparison

The following code performs a comparison between Value Iteration and Policy Iteration, by plotting the accumulated rewards of each episode with iterations in range $[1, 50]$ (might take a long time if not optimizied via numpy). You can perform all the test on a different versions of the environment, but with the same structure: *HugeLavaFloor*.

The function **run_episode(envirnonment, policy, max_iteration)** run an episode on the given environment using the input policy.

In [ ]:
env_name = "LavaFloor-v0"
#env_name = "HugeLavaFloor-v0"

print("Transition model for ", env_name, " : ") #assume transition functions is the same for all states
state=0
next_state=1
for i in range(0,env.action_space.n):
    print("probability of reaching ", env.state_to_pos(next_state), "from ", env.state_to_pos(state), " executing action ", env.actions[i], " : ", env.T[state, i, next_state])
print("Reward for non terminal states: ",env.RS[env.pos_to_state(0,0)]) #assume all states have same reward
for state in range(0,env.observation_space.n):
    if env.grid[state] == "P" or env.grid[state] == "G":
                    print("Reward for state :", env.state_to_pos(state) ," (state type: ", env.grid[state],") : ",env.RS[state])



maxiters = 49

env = gym.make(env_name)

series = []  # Series of learning rates to plot
liters = np.arange(maxiters + 1)  # Learning iteration values
liters[0] = 1
elimit = 100  # Limit of steps per episode
rep = 10  # Number of repetitions per iteration value
virewards = np.zeros(len(liters))  # Rewards array
c = 0

t = timer()

# Value iteration
for i in tqdm(liters, desc="Value Iteration", leave=True):
    reprew = 0
    policy = value_iteration(env, maxiters=i)  # Compute policy
        
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    virewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": virewards, "ls": "-", "label": "Value Iteration"})


vmaxiters = 5  # Max number of iterations to perform while evaluating a policy
pirewards = np.zeros(len(liters))  # Rewards array
c = 0

# Policy iteration
for i in tqdm(liters, desc="Policy Iteration", leave=True):
    reprew = 0
    policy = policy_iteration(env, maxiters=i)  # Compute policy
    # Repeat multiple times and compute mean reward
    for _ in range(rep):
        reprew += run_episode(env, policy, elimit)  # Execute policy
    pirewards[c] = reprew / rep
    c += 1
series.append({"x": liters, "y": pirewards, "ls": "-", "label": "Policy Iteration"})

print("Execution time: {0}s".format(round(timer() - t, 4)))
np.set_printoptions(linewidth=10000)

plot(series, "Learning Rate", "Iterations", "Reward")

Correct results for comparison can be found here below. Notice that since the executions are stochastic the charts could differ: the important thing is the global trend and the final convergence to an optimal solution.

**Standard Lava floor results comparison**
<img src="images/results-standard.png" width="600">

**Huge Lava floor results comparison** 
<img src="images/results-huge.png" width="600">